<a href="https://colab.research.google.com/github/jaimovi/Analise-de-similaridade-de-Banco-de-Perguntas-por-metodos-NLP/blob/main/Perguntas_ver1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Perguntas
##Este notebook estrutura inicialmente todas as etapas de pre processamento necessárias e que deverão ser aplicadas, independente de qual modelo . Estabelecer as etapas de transformação em caixa baixa, retirar stopwords, caracteres especiais e etc.Em seguida utilizaremos ainda neste Notebook o método de comparação por bag of words, calculando os vetores da sentenção, limitando o número de linhas, considerando a limitação de recursos computacionais do Colab. A opção então foi de importar todas as linhas do Banco de perguntas proposto, com 170000 perguntas e apenas buscar similaridades com as primeiras 5000 perguntas.





Carrego as Bibliotecas

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
# Load libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from keras.preprocessing import text, sequence
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

#Importar a base de Perguntas, Limitando a 5000 linhas iniciais.

In [ ]:
# Load Data
df=pd.read_excel('3 colunas de PERGUNTA.xlsx', usecols = "A,C:C")
df= df.rename(columns={'Texto da Pergunta': 'Pergunta'})
df= df.rename(columns={'Codigo da Pergunta': 'Codigo'})
df= df[:5000]

cabeçalho

In [ ]:
#Examinando a Importação
#Observar que toda pergunta possui um código unico, oriundo da base de dados.
df.head()

In [ ]:
df.shape

In [ ]:
df.tail(1)

In [ ]:
df.info()

# Exploratory Data Analysis

In [ ]:
#Há muita incidência de perguntas em Caixa alta e caixa baixa.
print(df.Pergunta)

##Transformar a coluna em análise Pergunta para minusculo, eliminar os valores nulos, stopwords, etc.

In [ ]:
auxiliar = df['Pergunta'].str.lower()
df['Pergunta']= auxiliar
df.head(1000)

In [ ]:
df.dropna(inplace=True)

In [ ]:
import unicodedata

def remove_accents(input_str):
    try:
      nfkd_form = unicodedata.normalize('NFKD', input_str)
    except:
      print(input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return str(only_ascii)

auxiliar = [remove_accents(sentence).replace('b\'','') for sentence in df['Pergunta']]
auxiliar2 = [sentence.replace('\'','') for sentence in auxiliar]
df['Pergunta']= auxiliar2

In [ ]:
df.head()

In [ ]:
todas_stopwords = stopwords.words('portuguese')
todas_stopwords.append('para')

In [ ]:
from nltk.corpus import stopwords
# inserir nova stopword
# stopwords.append('existe')
print(stopwords.words('portuguese'))

In [ ]:
# PROCURAR TEXTO PARA REMOVER STOPWORDS





In [ ]:
# df = df.explode("Pergunta", ignore_index=True)
# df.rename(columns={"Codigo": "Pergunta"}, inplace=True)
# df.index.name = "Code_ID"
# df.head()
# df.to_csv("Perguntas Tokenized.csv")


#Para o caso de bag of words:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# https://stackoverflow.com/questions/8897593/how-to-compute-the-similarity-between-two-text-documents
vect = TfidfVectorizer(min_df=1, stop_words=todas_stopwords)
tfidf = vect.fit_transform(df['Pergunta'][:5000])
pairwise_similarity = tfidf * tfidf.T

#Estamos buscando os pares de pergunta similares, de acordo com o cálculo das diferenças entre os cosenos, buscando um limiar adequado.

In [ ]:
pairwise_similarity

In [ ]:
pd.DataFrame(pairwise_similarity.toarray())

In [ ]:
#Armazenando o resultado em um Array
resultado = pd.DataFrame(pairwise_similarity.toarray())

In [ ]:
import numpy as np
import scipy
from scipy import stats

In [ ]:
print (resultado)
  
#?? print ("\nclipped arr1 : \n", stats.threshold(resultado, threshmin = 0.7 , threshmax = 1, newval = 0.85))

In [ ]:
resultado.head()

In [ ]:
resultado.tail()

In [ ]:
resultado.to_excel('resultado.xlsx')

In [ ]:
df['Pergunta'][86], df['Pergunta'][514],df['Pergunta'][133], df['Pergunta'][112]

In [ ]:
# Definir um valor de threshold para classificar como semelhante


# Remover as perguntas semelhantes do conjunto


In [ ]:

resultado[(resultado > 0.99)].any(1)

In [ ]:
to_drop = []
par_perguntas =[]

for column in resultado.columns:
 for index in resultado.index:
  if resultado.loc[index, column] > 0.95 and index != column:
   to_drop.append([index,column])
   par_perguntas.append([df['Pergunta'][index],df['Pergunta'][column]])

In [ ]:
#Identificamos (1110/2) 555 pares com semelhança de 0.95, sugerindo que sejam removidas do banco por duplicidade.
len(to_drop)

In [ ]:
#Identificando os pares
print(to_drop)

In [ ]:
to_drop

In [ ]:
par_perguntas[55]

In [ ]:
#Identificação textual dos pares de perguntas similares
par_perguntas

In [ ]:
#identificação textual pontual entre pares
df['Pergunta'][3779],df['Pergunta'][3914]
                                  

# Segunda parte do trabalho abordará a busca da similaridade entre as perguntas utilizando o modelo BERT para comparar os resultados de encontrados anteriormente na proposta  da etapa de Bag of words
##Aplicação de modelo transformer para similaridade


https://huggingface.co/neuralmind/bert-base-portuguese-cased



In [ ]:
import os
! apt-get update -qq > /dev/null   
# Install java
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
!pip install nlu  pyspark==2.4.7 > /dev/null   

import nlu

# 2.1 Carregar o dataset de Perguntas

## Estamos alterando os nomes da coluna para manter a compatibilidade com o Modelo proposto para o BERT

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


# Load dataset to Pandas
df= pd.read_excel('3 colunas de PERGUNTA.xlsx', usecols = "A,C:C")
df= df.rename(columns={'Texto da Pergunta': 'Title'})
df= df.rename(columns={'Codigo da Pergunta': 'Id'})
df= df[:5000]
max_r = 5000
df = df.iloc[0:max_r]
df

In [ ]:
auxiliar = df['Title'].str.lower()
df['Title']= auxiliar
df.head(1000)

In [ ]:
df.dropna(inplace=True)

In [ ]:
import unicodedata

def remove_accents(input_str):
    try:
      nfkd_form = unicodedata.normalize('NFKD', input_str)
    except:
      print(input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return str(only_ascii)

auxiliar = [remove_accents(sentence).replace('b\'','') for sentence in df['Title']]
auxiliar2 = [sentence.replace('\'','') for sentence in auxiliar]
df['Title']= auxiliar2

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
todas_stopwords = stopwords.words('portuguese')
todas_stopwords.append('para')

In [ ]:
from nltk.corpus import stopwords
# inserir nova stopword
# stopwords.append('existe')
print(stopwords.words('portuguese'))

# 2.2. Incorporar perguntas com embeddings de frases de BERT 

Podemos incorporar o título ou o corpo da pergunta.

In [ ]:
pipe = nlu.load('embed_sentence.bert')
#pipe = nlu.load('albert bert elmo electra xlnet pos')
predictions = pipe.predict(df.Title, output_level='document')
predictions

In [ ]:
predictions.iloc[0,2]


In [ ]:
predictions.iloc[0,1]

# 2.3 Como encontrar N sentenças mais semelhantes em um conjunto de dados para uma determinada sentença no conjunto de dados usando BERT
Frases com pequenas distâncias entre seus embeddings serão consideradas semelhantes entre si.

In [ ]:
## Calculate dinstance between all pairs of sentences in DF 
e_col = 'sentence_embedding_bert'

def get_sim_df_for_iloc(sent_iloc, predictions=predictions,e_col=e_col, pipe=pipe):
  # This function calculatse the distances for one sentences at  predictions[sent_iloc] to all other sentences in predictions using the embedding defined by e_col 
  # put embeddings in matrix
  embed_mat = np.array([x for x in predictions[e_col]])
  # calculate distance between every embedding pair
  sim_mat = cosine_similarity(embed_mat,embed_mat)
  print("Perguntas semelhantes a : " + df.iloc[sent_iloc].Title)
  # write sim scores to df
  df['sim_score'] = sim_mat[sent_iloc]
  return df 
sentence_to_compare=  19
to_drop_bert = []
par_perguntas_bert =[]

sim_df_for_one_sent = get_sim_df_for_iloc(sentence_to_compare,predictions,e_col)
sim_df_for_one_sent = sim_df_for_one_sent[sim_df_for_one_sent['sim_score']>0.97]
sim_df_for_one_sent.sort_values('sim_score', ascending = False)

# 2.4 Função de plotagem para traçar a distância entre uma frase no conjunto de dados e todas as outras frases.

In [ ]:
def viz_sim_df_for_one_sent( sent_iloc=0, N = 514, sim_df_for_one_sent=sim_df_for_one_sent):
  # Representa as N sentenças mais semelhantes em nosso dataframe para a sentença na posição sent_iloc
  sim_df_for_one_sent = get_sim_df_for_iloc(sent_iloc)
  
  sim_df_for_one_sent.index = sim_df_for_one_sent.Title
  sent = sim_df_for_one_sent.iloc[sent_iloc].Title
  ax = sim_df_for_one_sent.sort_values('sim_score', ascending = False).iloc[:N].sim_score.plot.barh(title=f'As {N} frases mais semelhantes em nosso conjunto de dados para a pergunta \ n"{sent}"', figsize=(40, 30))
  ax.set_xlim(0.8, 1)

# Basta inserir qualquer número e obter o gráfico de semelhanças da frase em df.iloc [i]
# No caso escolhemos a mesma pergunta do título 2.3
viz_sim_df_for_one_sent(19)

#2.5 Calcular cada pontuação de similaridade entre cada frase no dataframe de entrada em pares e gere a matriz de similaridade

In [ ]:
def get_sim_df_total( predictions,e_col, string_to_embed,pipe=pipe):
  # Esta função calcula as distâncias entre cada par de perguntas. Cria para sempre a frase uma nova coluna com o nome igual à frase com a qual ela se compara

 
  # put embeddings in matrix
  embed_mat = np.array([x for x in predictions[e_col]])
  # calcular a distância entre cada par de embedding
  sim_mat = cosine_similarity(embed_mat,embed_mat)
  # for i,v in enumerate(sim_mat): predictions[str(i)+'_sim'] = sim_mat[i]
  for i,v in enumerate(sim_mat): 
    s = predictions.iloc[i].document
    predictions[s] = sim_mat[i]

  return predictions 
sim_matrix_df = get_sim_df_total(predictions,'sentence_embedding_bert', sentence_to_compare )
sim_matrix_df

#2.6 Plotar mapa de calor da matriz de similaridade para as primeiras N perguntas

In [ ]:
non_sim_columns  = ['text','document','sentence_embedding_bert']

def viz_sim_matrix_first_n(num_sentences=5, sim_df = sim_matrix_df):
  # Plotar mapa de calor para as primeiras num_sentences
  fig, ax = plt.subplots(figsize=(20,10)) 
  sim_df.index = sim_df.document
  sim_columns = list(sim_df.columns)
  for b in non_sim_columns : sim_columns.remove(b)
  # sim_matrix_df[sim_columns]
  ax = sns.heatmap(sim_df.iloc[:num_sentences][sim_columns[:num_sentences]]) 

  ax.axes.set_title(f"Matriz de Similaridade para as {num_sentences} sentenças no dataset",)

viz_sim_matrix_first_n()

#2.7 Plotar mapa de calor da matriz de similaridade para as perguntas entre starT_iloc e end_iloc

In [ ]:
def viz_sim_matrix_from_to(start_iloc,end_iloc, sim_df = sim_matrix_df):
  # Plotar matriz térmica para sentenças em df.iloc [início: fim]   
  fig, ax = plt.subplots(figsize=(50,30)) 
  sim_df.index = sim_df.document
  sim_columns = list(sim_df.columns)
  for b in non_sim_columns : sim_columns.remove(b)


  ax = sns.heatmap(sim_df.iloc[start_iloc:end_iloc][sim_columns[start_iloc:end_iloc]]) # +2 because first 2 cols are not sim_scores

  ax.axes.set_title(f"Matriz de similaridade para as perguntas nas posições df.iloc[{start_iloc}:{end_iloc}] no dataset",)

viz_sim_matrix_from_to(10,20)

#2.8 Encontrar as N perguntas mais semelhantes em um conjunto de dados para uma nova pergunta que não existe nos dados usando BERT

In [ ]:
def get_sim_df_for_string(predictions,e_col, string_to_embed,pipe=pipe):
  # Cria um Dataframe que tem uma coluna sim_score que descreve a semelhança com a variável string_to_embed

  # colocar vetores de predições na matriz
  embed_mat = np.array([x for x in predictions[e_col]])

  # inserir string de entrada
  embedding = pipe.predict(string_to_embed).iloc[0][e_col]

  # Replicar incorporação para string de entrada
  m = np.array([embedding,]*len(df))
  sim_mat = cosine_similarity(m,embed_mat)

  #write sim score
  df['sim_score'] = sim_mat[0]


  return df

In [ ]:
question = ' contribui para o seguro social?'
sim_df = get_sim_df_for_string(predictions,'sentence_embedding_bert', 'existe ASO disponivel' )
ax = sim_df.sort_values('sim_score', ascending = False).iloc[:20][['sim_score','Title']].plot.barh(title = f"Frases mais semelhantes para pergunta\n'{question}'", figsize=(40,32))
ax.set_xlim(0.8, 1)

#2.8 Defina a função de plotagem do Helper para representar os resultados da incorporação de uma string

In [ ]:
def viz_sim_df_for_one_sent( question='foi feita a vistoria no andaime', e_col='sentence_embedding_bert', N = 40, sim_df_for_one_sent=sim_df_for_one_sent):
  # Plots the N most similar sentences in our dataframe for sentence at position sent_iloc
  sim_df = get_sim_df_for_string(predictions,e_col,question )
  sim_df.index = sim_df.Title
  sim_df.sort_values('sim_score', ascending = False).iloc[:N][['sim_score','Title']].plot.barh(title = f"Most similar Sentences for sentence\n'{question}'", figsize=(20,14))
  ax.set_xlim(0.8, 1)

In [ ]:
question = 'contribui para o seguro social?'
e_col = 'sentence_embedding_bert'
viz_sim_df_for_one_sent(question,e_col)

In [ ]:
viz_sim_df_for_one_sent('seguro social?')

# 2.9 Multi Embedding Similarity, encontre as N perguntas mais semelhantes em um conjunto de dados para uma nova frase usando BERT, USE, Electra

Primeiro, vamos carregar 3 embeddings ao mesmo tempo e incorporar o texto em nosso conjunto de dados.

In [ ]:
multi_pipe = nlu.load('use en.embed_sentence.electra embed_sentence.bert')
multi_embeddings = multi_pipe.predict(df.Title,output_level='document')
# multi_embeddings = multi_pipe.predict(df.Title)

multi_embeddings

#2.10 Cálculo de similaridade de Multi Embeddings


Vamos definir uma função que recebe uma string para incorporar, uma lista de embeddings e um pipeline

get_sim_df_for_string_multi () calcula todos os embeddings carregados no pipeline NLU de entrada para a string de entrada e calcula as distâncias para cada frase no DF de entrada em todos os embeddings e nos dará uma pontuação normalizada final.  

In [ ]:
def get_sim_df_for_string_multi(predictions,embed_col_names, string_to_embed,pipe=multi_pipe):
  # Cria um Dataframe que tem uma coluna sim_score que descreve a semelhança com a variável string_to_embed
  #Isso acumula as distâncias de todos os embeddings em embed_col_names e normaliza dividindo por len (embed_col_names)
  #faça uma matriz de semelhança vazia que irá armazenar as semelhanças agregadas entre diferentes embeddings
  predictions.dropna(inplace=True)
  cum_sim = np.zeros((len(predictions),len(predictions)))

  # embed with all embedders currently loaded in pipeline
  embeddings = pipe.predict(string_to_embed).iloc[0]

  #loop over all embeddings columns and accumulate the pairwise distances with string_to_embed into cum_sim
  for e_col in embed_col_names:
    # get the current embedding for input string
    embedding = embeddings[e_col]  
    # stack embedding vector for input string
    m = np.array([embedding,]*len(predictions)) 
    # put df vectors in np matrix
    embed_mat = np.array([x for x in predictions[e_col]]) 
    # calculate new similarities
    sim_mat = cosine_similarity(m,embed_mat) 
  # accumulate new simmilarities in cum_sum
    cum_sim += sim_mat  

  predictions['sim_score'] = cum_sim[0]/len(embed_col_names) 
  return predictions

In [ ]:
question = 'contribui para o seguro social?'
col_names = ['sentence_embedding_electra','sentence_embedding_bert', 'sentence_embedding_use']

sim_df = get_sim_df_for_string_multi(multi_embeddings,col_names, question )
sim_df.index = sim_df.document
sim_df.sort_values('sim_score', ascending = False).iloc[:15][['sim_score','document']].plot.barh(title = f"Most similar Sentences for sentence\n'{question}'", figsize=(20,14))

# 2.11 Defina a função auxiliar para traçar os resultados de similaridade de uma string multiincorporada

In [ ]:
def viz_sim_df_for_one_sent_multi_embed( question='Foi feita a vistoria no Guindaste', e_cols=col_names, N = 40, multi_embeddings=multi_embeddings):
  # Representa as N sentenças mais semelhantes em nosso dataframe para a sentença na posição sent_iloc
  sim_df = get_sim_df_for_string_multi(multi_embeddings,col_names, question )
  sim_df.index = sim_df.document
  sim_df.sort_values('sim_score', ascending = False).iloc[:N][['sim_score','document']].plot.barh(title = f"Most similar Sentences for sentence\n'{question}'",figsize=(20,14))

  ax.set_xlim(0.8, 1)

question = 'contribui para o seguro social'
col_names = ['sentence_embedding_electra','sentence_embedding_bert', 'sentence_embedding_use']
viz_sim_df_for_one_sent_multi_embed(question, col_names)

In [ ]:
question = 'ASO'
viz_sim_df_for_one_sent_multi_embed(question)

In [ ]:
question = 'treinamento'
viz_sim_df_for_one_sent_multi_embed(question)

In [ ]:
#from transformers import AutoTokenizer, AutoModelForMaskedLM
  
#tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

#model = AutoModelForMaskedLM.from_pretrained("neuralmind/bert-base-portuguese-cased")